<a href="https://colab.research.google.com/github/fortune901/nubase-project/blob/main/project1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [48]:
#!git clone https://github.com/fortune901/nubase-project.git # cloner mon depot github
#%cd nubase-project # va dans le dossier du depot
#!ls #liste les fichiers pour verifier
from google.colab import files
uploaded = files.upload()

Saving nubase_4.mas20 to nubase_4 (11).mas20


In [55]:

#importation des bibliothèques nécessaires

import pandas as pd # pour lire et manipuler les données tabulaires (fichier.csv)
from sklearn.model_selection import train_test_split # pour diviser les données en ensemble d'entrainement et de test
from sklearn.preprocessing import StandardScaler # pour normaliser les données
from tensorflow.keras.models import Sequential # ici c'est pour construire un modèle de reseau de neurone
from tensorflow.keras.layers import Dense # ici c'est pour pouvoir ajouter des couches de neurones cachees
#import re
# Fonction de conversion d'une demi-vie texte en secondes
#def convertir_demi_vie_en_seconndes(texte):
  #"""
  #Convertit une demi-vie sous forme de texte (ex. '2.5 ms) en secondes.
  #"""
  #if not texte or texte.lower() =='stbl':
   # return None
    #texte = texte.strip().replace('-', '-').replace('µ', 'u') # Nettoyage
    #match = re.match(r'^([0-9.eE+\-]+)\s*([a-zA-Z]+)$', texte)
    #if not match:
     # return None
    #valeur, unite = match.groups()
    #try:
      #valeur = float(valeur)
    #except ValueError:
      #return None
      #facteurs ={'s': 1,
                 #'ms': 1e-3,
                 #'us': 1e-6,
                 #'ns': 1e-9,
                 #'ps': 1e-12,
                 #'m': 60,
                 #'h': 3600,
                 #'d': 86400,
                 #'y': 365.25* 86400,
                 #'ky': 1e3* 365.25* 86400,
                 #'My': 1e6* 365.25* 86400,
                 #'Gy': 1e9* 365.25* 86400,
                 #}
    #unite = unite.lower()
    #if unite not in facteurs:
      #return None
      #return valeur * facteurs[unite]
import re

def convertir_demi_vie_en_secondes(demi_vie_texte):
    if not demi_vie_texte or demi_vie_texte.strip() == '':
        return None

    # Dictionnaire des unités vers secondes
    conversion = {
        's': 1,
        'ms': 1e-3,
        'us': 1e-6,
        'ns': 1e-9,
        'min': 60,
        'h': 3600,
        'H': 3600,
        'd': 86400,
        'y': 31536000,
        'a': 31536000,  # parfois y = a = année
    }

    # Extraction avec expression régulière
    match = re.match(r'([\d\.]+)\s*([a-zA-Zµ]+)', demi_vie_texte.strip())
    if match:
        valeur = float(match.group(1))
        unite = match.group(2).lower()  # on passe l’unité en minuscule

        for cle in conversion:
            if cle.lower() == unite:
                return valeur * conversion[cle]

    return None  # si non reconnu
#Extraction des donnees
def extraire_donnees_nubase(fichier):
  """
  Extrait les donnees nucleaires depuis le fichier nubase_4.mas20.
  """
  donnees = []
  with open(fichier, 'r', encoding='utf-8') as f:
    for ligne in f:
      if ligne.strip() == '' or ligne.startswith('#'): # cette ligne de code permet de sauter les lignes vides ou de commmentaire
        continue
      try:
        #Lectures des champs depuis les positions fixes (selon la structure de nubase2020)
        # attention : les indices peuvent varier selon les versions du fichier
        Z = ligne[0:3].strip() # Numero atomique (proton)
        N = ligne[4:8].strip() # Nombre de neutrons
        A = ligne[9:13].strip() # Nombre de masse
        element = ligne[14:17].strip() # Symbole de l'element
        etat= ligne[18:19].strip() # état fonfamentale (g) ou isomere (m)
        demi_vie_texte = ligne[59:80].strip() # Demi-vie (au format texte, souvent log ou unité ex: '1.23 s')
        if demi_vie_texte.lower() =='stbl': #ici on saute les noyaux stable
            continue
          #demi_vie_s = None # Initialisation par defaut
        try:
          demi_vie_s = convertir_demi_vie_en_secondes(demi_vie_texte) # convertir en secondes
        except:
            demi_vie_s = None
            pass  # ignore l'erreur si ca ne convertit pas
        deformation= ligne[90:100].strip() # Deformation nucleaire (si disponible)
        shell_effect = ligne[100:110].strip() # Effet de coquille (ex: energie de liaison)
        spin_orbite = ligne[110:120].strip() # Info sur le spin ou couplage (selon le format exact)
        # Ajoutons les données sous formes de dictionnaire

        donnees.append({'Z': Z,
                       'N': N,
                       'A': A,
                       'Element': element,
                       'Etat': etat,
                       'Demi-vie (texte)': demi_vie_texte,
                        'Demi-vie (s)': demi_vie_s, #ajout de la demi-vie en secondes
                       'Deformation': deformation,
                       'Shell_effect': shell_effect,
                       'Spin_orbite': spin_orbite})
      except IndexError: # si une ligne est trop courte on l'ignore
        continue
  return donnees
# Utilisation du script
fichier = 'nubase_4.mas20'
donnees = extraire_donnees_nubase(fichier)
#affichage des 5 premiers lignes extraites
for ligne in donnees[:5]:
  print(ligne)
# Utilisation du script
fichier = 'nubase_4.mas20'
donnees = extraire_donnees_nubase(fichier)
#affichage de toutes les lignes extraites
for ligne in donnees:
  print(ligne)


{'Z': '001', 'N': '0000', 'A': '1n', 'Element': '', 'Etat': '', 'Demi-vie (texte)': '609.8    s', 'Demi-vie (s)': 609.8, 'Deformation': '2+*', 'Shell_effect': '06', 'Spin_orbite': '1932 B'}
{'Z': '003', 'N': '0010', 'A': '3H', 'Element': '', 'Etat': '', 'Demi-vie (texte)': '12.32   y', 'Demi-vie (s)': 388523520.0, 'Deformation': '2+*', 'Shell_effect': '00', 'Spin_orbite': '1934 B'}
{'Z': '003', 'N': '0030', 'A': '3L', 'Element': '', 'Etat': '', 'Demi-vie (texte)': 'p-unst', 'Demi-vie (s)': None, 'Deformation': '2-#', 'Shell_effect': '98', 'Spin_orbite': 'p'}
{'Z': '004', 'N': '0010', 'A': '4H', 'Element': '', 'Etat': '', 'Demi-vie (texte)': '139     ys', 'Demi-vie (s)': None, 'Deformation': '', 'Shell_effect': '98', 'Spin_orbite': '1981 n'}
{'Z': '004', 'N': '0030', 'A': '4L', 'Element': '', 'Etat': '', 'Demi-vie (texte)': '91     ys', 'Demi-vie (s)': None, 'Deformation': '', 'Shell_effect': '98', 'Spin_orbite': '1965 p'}
{'Z': '001', 'N': '0000', 'A': '1n', 'Element': '', 'Etat': '', 

In [52]:
import csv
# Enregistrement dans un fichier CSV
with open('donnees_nubase.csv', 'w', newline='', encoding='utf-8') as csvfile:
  champs = ['Z', 'N','A','Element',  'Etat', 'Demi-vie (texte)', 'Demi-vie (s)', 'Deformation', 'Shell_effect', 'Spin_orbite']
  writer = csv.DictWriter(csvfile, fieldnames=champs)
  writer.writeheader()
  writer.writerows(donnees)

# cONVERTIR LA LISTE DE DICTIONNAIRES EN DATAFRAME
df = pd.DataFrame(donnees)
df
df.to_csv("donnees_nubase.csv", index=False)
df[['N','Z', 'Demi-vie (s)']]
from google.colab import files
files.download("donnees_nubase.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>